<a href="https://colab.research.google.com/github/Rajeshwari125/Flights-Booking-simulator/blob/main/Copy_of_Welcome_to_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab cell
!pip install -q fastapi uvicorn sqlmodel


In [ ]:

!sed -n '1,240p' main.py


from typing import List, Optional
from datetime import datetime, date, timedelta

from fastapi import FastAPI, HTTPException, Query, Depends
from sqlmodel import SQLModel, Field, create_engine, Session, select

DATABASE_URL = "sqlite:///./flights.db"
engine = create_engine(DATABASE_URL, echo=False)

class Flight(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    airline: str
    origin: str = Field(index=True)
    destination: str = Field(index=True)
    departure_time: datetime
    arrival_time: datetime
    price: float
    available_seats: int

    @property
    def duration(self) -> int:
        delta = self.arrival_time - self.departure_time
        return int(delta.total_seconds() // 60)

app = FastAPI(title="Flight Search API - Milestone 1")

def create_db_and_tables():
    SQLModel.metadata.create_all(engine)

def populate_sample_data_if_empty():
    with Session(engine) as session:
        existing = session.exec(select(Flight)).first()
  

In [ ]:
%%writefile main.py
from typing import List, Optional
from datetime import datetime, date, timedelta

from fastapi import FastAPI, HTTPException, Query, Depends
from sqlmodel import SQLModel, Field, create_engine, Session, select

DATABASE_URL = "sqlite:///./flights.db"
engine = create_engine(DATABASE_URL, echo=False)

class Flight(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    airline: str
    origin: str = Field(index=True)
    destination: str = Field(index=True)
    departure_time: datetime
    arrival_time: datetime
    price: float
    available_seats: int

    @property
    def duration(self) -> int:
        delta = self.arrival_time - self.departure_time
        return int(delta.total_seconds() // 60)

app = FastAPI(title="Flight Search API - Milestone 1")

def create_db_and_tables():
    SQLModel.metadata.create_all(engine)

def populate_sample_data_if_empty():
    with Session(engine) as session:
        existing = session.exec(select(Flight)).first()
        if existing:
            return

        sample = [
            Flight(
                airline="BlueAir",
                origin="Chennai",
                destination="Bengaluru",
                departure_time=datetime(2025,12,1,6,30),
                arrival_time=datetime(2025,12,1,7,45),
                price=3200, available_seats=50
            ),
            Flight(
                airline="SkyFly",
                origin="Chennai",
                destination="Mumbai",
                departure_time=datetime(2025,12,1,9,0),
                arrival_time=datetime(2025,12,1,10,50),
                price=4200, available_seats=20
            ),
        ]

        session.add_all(sample)
        session.commit()

@app.on_event("startup")
def startup():
    create_db_and_tables()
    populate_sample_data_if_empty()

def get_session():
    with Session(engine) as session:
        yield session

@app.get("/flights", response_model=List[Flight])
def list_flights(session: Session = Depends(get_session)):
    return session.exec(select(Flight)).all()

@app.get("/search", response_model=List[Flight])
def search_flights(
    origin: Optional[str] = None,
    destination: Optional[str] = None,
    session: Session = Depends(get_session)
):
    stmt = select(Flight)
    if origin:
        stmt = stmt.where(Flight.origin == origin)
    if destination:
        stmt = stmt.where(Flight.destination == destination)
    return session.exec(stmt).all()


Overwriting main.py


In [20]:
# Test inside Colab using TestClient - run in a NEW cell (not inside %%writefile)
from fastapi.testclient import TestClient
import importlib
import main

importlib.reload(main)   # ensures the newly written file is loaded
client = TestClient(main.app)

resp = client.get("/flights")
print("Status:", resp.status_code)
print("Flights:", resp.json())


/usr/local/lib/python3.12/dist-packages/sqlmodel/main.py:644: SAWarning: This declarative base already contains a class with the same class name and module name as main.Flight, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)


InvalidRequestError: Table 'flight' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [21]:
import sqlite3

conn = sqlite3.connect('flights.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM flight")

# Get column names
column_names = [description[0] for description in cursor.description]
print("Columns:", column_names)

# Fetch all rows
rows = cursor.fetchall()
print("Rows:")
for row in rows:
    print(row)

conn.close()

Columns: ['id', 'airline', 'origin', 'destination', 'departure_time', 'arrival_time', 'price', 'available_seats']
Rows:
(1, 'BlueAir', 'Chennai', 'Bengaluru', '2025-12-01 06:30:00.000000', '2025-12-01 07:45:00.000000', 3200.0, 50)
(2, 'SkyFly', 'Chennai', 'Mumbai', '2025-12-01 09:00:00.000000', '2025-12-01 10:50:00.000000', 4200.0, 20)


In [ ]:
!rm -f flights.db


In [ ]:
import importlib, main
main.SQLModel.metadata.clear() # Clear existing table definitions
importlib.reload(main)

/usr/local/lib/python3.12/dist-packages/sqlmodel/main.py:644: SAWarning: This declarative base already contains a class with the same class name and module name as main.Flight, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)


<module 'main' from '/content/main.py'>

In [19]:
from fastapi.testclient import TestClient
client = TestClient(main.app)

main.create_db_and_tables()
main.populate_sample_data_if_empty()

resp = client.get("/flights")
print(resp.status_code)
print(resp.json())

200
[{'departure_time': '2025-12-01T06:30:00', 'airline': 'BlueAir', 'id': 1, 'origin': 'Chennai', 'price': 3200.0, 'destination': 'Bengaluru', 'arrival_time': '2025-12-01T07:45:00', 'available_seats': 50}, {'departure_time': '2025-12-01T09:00:00', 'airline': 'SkyFly', 'id': 2, 'origin': 'Chennai', 'price': 4200.0, 'destination': 'Mumbai', 'arrival_time': '2025-12-01T10:50:00', 'available_seats': 20}]
